In [9]:
import pandas as pd
from finvizfinance.screener.overview import Overview

def get_combined_top_stocks(countries=['USA', 'Canada'], limit_per_country=50):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching top rated stocks for: {country}...")
        
        try:
            foverview = Overview()
            
            # Use the correct filter syntax we found earlier
            filters_dict = {
                'Analyst Recom.': 'Strong Buy (1)',
                'Country': country
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- FIX STARTS HERE ---
    # 1. Inspect available columns to avoid guessing
    available_columns = combined_df.columns.tolist()
    print(f"\nDebug: Columns found in data: {available_columns}")

    # 2. Determine which 'Recommendation' column name is being used (if any)
    recom_col = None
    if 'Recom' in available_columns:
        recom_col = 'Recom'
    elif 'Analyst Recom' in available_columns:
        recom_col = 'Analyst Recom'
        
    # 3. Sort if we found the column
    if recom_col:
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # 4. safely build the list of columns to return
    # We only include columns that actually exist in 'available_columns'
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country']
    
    # Add the recommendation column to our list if it exists
    if recom_col:
        desired_cols.append(recom_col)

    # Filter columns safely
    final_cols = [c for c in desired_cols if c in available_columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_north_america = get_combined_top_stocks(['USA', 'Canada'])

if not top_north_america.empty:
    print(f"\nFound {len(top_north_america)} stocks.")
    print(top_north_america)
    
    # Get tickers
    ticker_list = top_north_america['Ticker'].tolist()
    print(f"\nTicker List: {ticker_list}")
else:
    print("DataFrame is empty.")

Fetching top rated stocks for: USA...
Fetching top rated stocks for: Canada...###########] 77/78 


C:\Users\jdcc3\AppData\Roaming\Python\Python313\site-packages\finvizfinance\screener\base.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)


[Info] loading page [########################------] 4/5 
Debug: Columns found in data: ['Ticker', 'Company', 'Sector', 'Industry', 'Country', 'Market Cap', 'P/E', 'Price', 'Change', 'Volume', 'Source_Country']

Found 100 stocks.
   Ticker                              Company              Sector  Price  \
0    AAON                             AAON Inc         Industrials  93.48   
1    AARD            Aardvark Therapeutics Inc          Healthcare   9.98   
2    ABAT  American Battery Technology Company         Industrials   3.74   
3    ABEO              Abeona Therapeutics Inc          Healthcare   5.07   
4     ABL         Abacus Global Management Inc           Financial   6.63   
..    ...                                  ...                 ...    ...   
95   AORT                         Artivion Inc          Healthcare  46.66   
96   AOUT          American Outdoor Brands Inc   Consumer Cyclical   7.17   
97     AP                Ampco-Pittsburgh Corp         Industrials   2.59   


In [10]:
import sys
!"{sys.executable}" -m pip install yfinance
import yfinance as yf

# Take the first 3 from our new list as an example
for symbol in ticker_list[:3]:
    stock = yf.Ticker(symbol)
    print(f"\n--- Recent Actions for {symbol} ---")
    
    # Get the latest upgrade/downgrade
    upgrades = stock.upgrades_downgrades
    if not upgrades.empty:
        # Show only actions from 2024/2025 (filtering by date)
        print(upgrades.tail(3))
    else:
        print("No recent analyst actions logged.")

Defaulting to user installation because normal site-packages is not writeable

--- Recent Actions for AAON ---
                            Firm       ToGrade FromGrade Action  \
GradeDate                                                         
2019-05-06 12:22:23  DA Davidson  Underperform   Neutral   down   
2016-10-13 04:00:00    Jefferies           Buy             init   
2015-02-02 21:42:06  DA Davidson       Neutral             init   

                    priceTargetAction  currentPriceTarget  priorPriceTarget  
GradeDate                                                                    
2019-05-06 12:22:23         Announces                35.0               0.0  
2016-10-13 04:00:00         Announces                 0.0               0.0  
2015-02-02 21:42:06         Announces                25.0               0.0  

--- Recent Actions for AARD ---
                                  Firm     ToGrade FromGrade Action  \
GradeDate                                                  